# Kari

Introduces the main ways of using Kari.

## Training

Import the neccecary modules

In [70]:
from dataset import Dataset
from utils_parameter_parsing import *
from sequence_processing_model import SequenceProcessingModel

Load the parameters from the config file and create a model. Note that you can also overide any arguments in `config.ini` by passing them in a dictionary to `process_parameters`.

In [71]:
config = config_parser('./config.ini') # parse config.ini
parameters = process_parameters(config, {}) # process_parameters expects a dictionary

Create a `SequenceProcessingModel` object. This object coordinates training/prediction/loading of models and datasets.

In [72]:
sequence_processer = SequenceProcessingModel(**parameters)

We can then load the dataset (specified by the `dataset_text_folder` parameter). If pretrained token embeddings were specified with `token_pretrained_embedding_filepath`, these will be loaded also.

In [4]:
sequence_processer.load_dataset()

Loading dataset... Done (0.71 seconds)
Loading embeddings... found 1891782 word vectors of dimension 200.
Done (72.29 seconds)


Lastly, we specify the model we would like to use. Models are specified by the `model_name` parameter.

In [5]:
sequence_processer.specify_model()

Building the simple LSTM-CRF model for NER...WARNING:tensorflow:From /home/john/miniconda3/envs/kari/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:1247: calling reduce_sum (from tensorflow.python.ops.math_ops) with keep_dims is deprecated and will be removed in a future version.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 50)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 50, 200)           2164000   
_______

In [6]:
sequence_processer.fit()

Train on 5712 samples, validate on 635 samples
Epoch 1/50
5712/5712 [==============================] - 201s 35ms/step - loss: 3.2505 - acc: 0.9045 - val_loss: 2.6433 - val_acc: 0.9262
Epoch 2/50
5712/5712 [==============================] - 200s 35ms/step - loss: 3.1122 - acc: 0.9289 - val_loss: 2.5873 - val_acc: 0.9425
Epoch 3/50
5712/5712 [==============================] - 200s 35ms/step - loss: 3.0688 - acc: 0.9410 - val_loss: 2.5575 - val_acc: 0.9518
Epoch 4/50
5712/5712 [==============================] - 200s 35ms/step - loss: 3.0411 - acc: 0.9486 - val_loss: 2.5399 - val_acc: 0.9553
Epoch 5/50
5712/5712 [==============================] - 200s 35ms/step - loss: 3.0218 - acc: 0.9527 - val_loss: 2.5242 - val_acc: 0.9598
Epoch 6/50
5712/5712 [==============================] - 200s 35ms/step - loss: 3.0076 - acc: 0.9557 - val_loss: 2.5140 - val_acc: 0.9634
Epoch 7/50
5712/5712 [==============================] - 200s 35ms/step - loss: 2.9969 - acc: 0.9600 - val_loss: 2.5067 - val_acc: 0

ValueError: DataFrame constructor not properly called!

## Loading a pretrained model

Loading a pre-trained model is easy. First, we create the SequenceProcessingModel object exactly as before

In [76]:
config = config_parser('./config.ini') # parse config.ini
parameters = process_parameters(config, {}) # process_parameters expects a dictionary

sequence_processer = SequenceProcessingModel(**parameters)

and then call `load_pretrained_model()` passing in the filepath to the model weights

In [ ]:
sequence_processer.load_pretrained_model('../output/model_checkpoint.50-2.47.hdf5')

## Evaluation

In [29]:
import numpy as np

from sklearn.metrics import precision_recall_fscore_support

In [30]:
pred_idx, gold_idx, labels_ = sequence_processer.predict()
np.count_nonzero((pred_idx == gold_idx)) / len(pred_idx)

0.9855531914893617

In [31]:
neg_tag = sequence_processer.ds.tag_type_to_index['O']
pred_labels = pred_idx != neg_tag
gold_labels = gold_idx != neg_tag

In [32]:
TP = 0
FP = 0
FN = 0

for pred, gold in zip(pred_labels, gold_labels):
    # FN
    if gold and not pred:
        FN += 1
    # FP
    if not gold and pred:
        FP += 1
    # TP
    if gold and pred:
        TP += 1
        
precision = TP / (TP + FP)
recall = TP / (TP + FN)
f1 = 2 * precision * recall / (precision + recall)
        
    

In [33]:
TP, FP, FN

(1612, 149, 416)

In [34]:
precision, recall, f1

(0.9153889835320841, 0.7948717948717948, 0.8508841382950645)

In [64]:
labels_ = [(k, v)[1] for k, v in sequence_processer.ds.tag_type_to_index.items() if k != 'O']

In [68]:
performance = precision_recall_fscore_support(gold_idx, pred_idx, average=None)

In [69]:
performance

(array([0.99080439, 0.88308977, 0.82672234, 0.83632287, 0.85714286]),
 array([0.99668683, 0.79661017, 0.74576271, 0.68190128, 0.73031026]),
 array([0.99373691, 0.83762376, 0.78415842, 0.75125881, 0.78865979]),
 array([44972,   531,   531,   547,   419]))

In [63]:
labels_

tag_to_idx = sequence_processer.ds.tag_type_to_index.items()

print("Precision\n{}:{}".format(tag_to_idx[labels_[0]]performance[0]))

[1, 2, 3, 4]

In [66]:
sequence_processer.ds.tag_type_to_index

{'B-Disease': 2, 'E-Disease': 1, 'I-Disease': 3, 'O': 0, 'S-Disease': 4}